@kafkoders Google HashCode 2014 Final Statement

##Car
Class representing a Street View Car

In [0]:
import pandas as pd
import numpy as np
import math
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
class Car:
  def __init__(self, index_, time_, current_junction):
    '''
    index_ -> int
    time_ -> int
    current_junction -> int
    '''
    self.index_ = index_
    self.time_ = time_
    self.current_junction = current_junction
    self.last_junction = None

In [0]:
class Junction:
  def __init__(self, index_, longitude_, latitude_):
    '''
    index_ -> int
    longitude_ -> float
    latitude_ -> float
    streets_ -> int[]
    '''
    self.index_ = index_
    self.longitude_ = longitude_
    self.latitude_ = latitude_
    self.streets_ = []

In [0]:
class Street:
  def __init__(self, index_, from_, to_, directions_, cost_, length_):
    '''
    index_ -> int
    from_ -> int
    to_ -> int
    directions_ -> int[1,2]
    cost_ -> int
    length_ -> int
    covered_ -> boolean
    '''
    self.index_ = index_
    self.from_ = from_
    self.to_ = to_
    self.directions_ = directions_
    self.cost_ = cost_
    self.length_ = length_
    self.covered_ = False

In [0]:
def create_graph_from_dataset(file_):
  flag = False
  num_streets = 0
  street_counter = 0
  num_junctions = 0
  junction_counter = 0
  time_ = 0
  num_cars = 0
  starting_node_index = 0
  streets_ = []
  junctions_ = []
  cars_ = []

  with open('/content/' + file_ + '.txt') as input_:
    for line in input_:
      if flag is False:
        num_junctions, num_streets, time_, num_cars, starting_node_index = line.split(' ')
        
        flag = True
      else:
        if junction_counter < int(num_junctions):
          #print(line)
          longitude_, latitude_ = line.split(' ')
          junctions_.append(Junction(junction_counter, float(longitude_), float(latitude_)))
          junction_counter += 1
        elif street_counter < int(num_streets):
          #print(str(junction_counter) + " | " + num_junctions)
          from_, to_, directions_, cost_, length_  = line.split(' ')
          tmp_street = Street(street_counter, int(from_), int(to_), int(directions_), int(cost_), int(length_))
          streets_.append(tmp_street)
          junctions_[int(from_)].streets_.append(street_counter)
          if int(directions_) == 2:
            junctions_[int(to_)].streets_.append(street_counter)
          street_counter += 1
    for i in range(int(num_cars)):
          cars_.append(Car(i, int(time_),int(starting_node_index)))

  return junctions_, streets_, cars_, time_

In [0]:
def plot_graph(junctions_):
  plt.title("Graph location")
  plt.xlabel("Longitude")
  plt.ylabel("Latitude")
  for junction_ in junctions_:
    plt.scatter(junction_.longitude_, junction_.latitude_)
  plt.show()

In [0]:
def get_best_street(car_, junction_streets, streets_):
  '''
  Algorthm that tries to retrieve the better option in order to maximize
  the distance covered by a car.
  '''
  tmp_street_not = None
  tmp_street_yes = None
  for street_ in junction_streets:
    if streets_[street_].covered_ is False and car_.time_ >= streets_[street_].cost_ and (tmp_street_not is None or tmp_street_not.length_ < streets_[street_].length_):
      tmp_street_not = streets_[street_]
    if streets_[street_].covered_ is True and car_.time_ >= streets_[street_].cost_ and \
    (tmp_street_yes is None or tmp_street_yes.cost_ > streets_[street_].cost_):
      if streets_[street_].from_ == car_.current_junction and \
      streets_[street_].to_ != car_.last_junction:
        tmp_street_yes = streets_[street_]
      elif streets_[street_].to_ == car_.current_junction and \
      streets_[street_].from_ != car_.last_junction:
        tmp_street_yes = streets_[street_]
  if tmp_street_not is not None:
    return tmp_street_not
  elif tmp_street_yes is not None:
    return tmp_street_yes
  else:
    for street_ in junction_streets:
      if streets_[street_].covered_ is True and car_.time_ >= streets_[street_].cost_ and \
      (tmp_street_yes is None or tmp_street_yes.cost_ > streets_[street_].cost_):
        tmp_street_yes = streets_[street_]
    return tmp_street_yes


In [0]:
def move_car(car_, street_, junction_):
  car_.last_junction = car_.current_junction
  if car_.current_junction == street_.from_:
    car_.current_junction = street_.to_
    car_.time_ -= street_.cost_
  else:
    car_.current_junction = street_.from_
    car_.time_ -= street_.cost_
  return car_

In [0]:
def move_fleet(cars_, junctions_, streets_, distance_):
  end_ = True
  for car_ in cars_:
    best_street = get_best_street(car_, junctions_[car_.current_junction].streets_, streets_)
    if best_street is not None:
      end_ = False
      tmp_car = move_car(car_, best_street, car_.current_junction)
      cars_.pop(tmp_car.index_)
      cars_.insert(tmp_car.index_, tmp_car)
      if best_street.covered_ is not True:
        streets_[best_street.index_].covered_ = True
        distance_ += best_street.length_
  return end_, cars_, streets_, distance_

In [10]:
if __name__ == '__main__':
  junctions_, streets_, cars_, time_ = create_graph_from_dataset('street_view.in')
  print("Data loaded")
  distance_ = 0

Data loaded


In [11]:
  iteration_ = 0
  while(True):
    exit_, cars_, streets_, distance_ = move_fleet(cars_, junctions_, streets_, distance_)
    iteration_ += 1
    if exit_ is True:
      max_distance = 0
      for street_ in streets_:
        max_distance += street_.length_
      print("Total distance covered: " + str(distance_) + " of: " + str(max_distance) + " | Number of iterations: " + str(iteration_))
      print("Remaining time and final position for each car")
      for car_ in cars_:
        print("Car number: " + str(car_.index_) + " | longitude: " + str(junctions_[car_.current_junction].longitude_) + "\t\tlatitude: " + str(junctions_[car_.current_junction].latitude_) + " and index: " + str(car_.current_junction))
      break

Total distance covered: 209769 of: 1967444 | Number of iterations: 32327
Remaining time and final position for each car
Car number: 0 | longitude: 48.874786		latitude: 2.3284404000000003 and index: 8363
Car number: 1 | longitude: 48.874692100000004		latitude: 2.3286390000000003 and index: 5518
Car number: 2 | longitude: 48.828772400000005		latitude: 2.3427007 and index: 5392
Car number: 3 | longitude: 48.8746367		latitude: 2.3284279000000003 and index: 7148
Car number: 4 | longitude: 48.8746367		latitude: 2.3284279000000003 and index: 7148
Car number: 5 | longitude: 48.8698032		latitude: 2.3223839 and index: 6125
Car number: 6 | longitude: 48.874692100000004		latitude: 2.3286390000000003 and index: 5518
Car number: 7 | longitude: 48.874692100000004		latitude: 2.3286390000000003 and index: 5518
